# DICE Model
**Exercise Session Resource Economics (Spring Term 2025)** \
Raul Hochuli (raul.hochuli@unibas.ch)


## Preamble

The following code has been replicated and refactored for this course based on the inital material from [Hazem Krichene](https://github.com/hazem2410) in his public Github repository [PYDICE](https://github.com/hazem2410/PyDICE), which builds on the [DICE (Dynamic Integrated Climate Economy) Model of William D. Nordhaus](https://williamnordhaus.com/dicerice-models). 


## Code

### Pre-requisits

The cells below contain the ususal pre-requisits needed for modelling, e.g. the packages, the function mounting Google Colab to Google Drive and a plotting function for easier visualization of all available model exports

In [2]:
# Cell should only apply when running this notebook on google COLAB
# > asks for access to Drive and permission to read and write files
path_where_notebook_is_stored_on_GDrive = 'MyDrive/1_UNIBAS_phd/ResEcon25/DICE'
import os as os

if 'content' in os.getcwd():
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir(f'/content/drive/{path_where_notebook_is_stored_on_GDrive}')


In [3]:
# Packages used in the notebook
import os as os
import numpy as np
import pandas as pd
import glob
import shutil
import datetime 
import scipy.optimize as opt
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots


In [14]:

def plot_comparison(output_dir_name = 'output', plot_dir_name = 'plots', title = 'DICE Economic and Emission variables', agg_file_name:str = 'DICE_agg', n_columns: int = 4, fig_indv_show: bool = True, fig_comp_show: bool = True, fig_export: bool = True):
    TT = np.linspace(2000, 2500, 100, dtype=np.int32)
    os.makedirs(plot_dir_name, exist_ok=True) if fig_export else None
    os.makedirs(f'{plot_dir_name}/single_plots', exist_ok=True) if fig_export else None


    # get all model results in output directory + cosncatenate to 1 df
    result_df_paths = sorted(glob.glob(f'{output_dir_name}/*.csv'))
    
    mod_comp_list = []
    for path in result_df_paths:
        df = pd.read_csv(path)
        mod_comp_list.append(df)

    mod_comp_df = pd.concat(mod_comp_list, axis=0)


    # model individual plots ------------------------------
    cols_to_plot = [col for col in mod_comp_df.columns if col not in ['scen_name', 't', 't_year']]
    n_rows = len(cols_to_plot) // n_columns + 1 if len(cols_to_plot) % n_columns != 0 else len(cols_to_plot) // n_columns

    for scen in mod_comp_df['scen_name'].unique():

        fig = make_subplots(rows=n_rows, cols=n_columns, subplot_titles=cols_to_plot)

        for i, col_plot in enumerate(cols_to_plot):
            row = i // n_columns + 1
            col = i % n_columns + 1
            fig.add_trace(go.Scatter(x=TT,
                                    y=mod_comp_df.loc[mod_comp_df['scen_name'] == scen, col_plot].values,
                                    mode='lines', name=col_plot), row=row, col=col)

        fig.update_layout(title_text=f"{title} ({scen})", template='plotly_white', showlegend=False)
        fig.write_html(f'{plot_dir_name}/{scen}.html') if fig_export else None
        fig.show() if fig_indv_show else None


    # model comparison plot ------------------------------
    color_scale = px.colors.sequential.Rainbow
    fig_comp_scen_legend   = make_subplots(rows=n_rows, cols=n_columns, subplot_titles=cols_to_plot)
    for i, col_plot in enumerate(cols_to_plot):
        
        fig_sing = go.Figure()

        for j, scen in enumerate(mod_comp_df['scen_name'].unique()):
            row = i // n_columns + 1
            col = i % n_columns + 1
            scen_color = color_scale[j % len(color_scale)]

            showlegend_TF = True if i == 0 else False
            fig_comp_scen_legend.add_trace(go.Scatter(x=TT,
                                                y=mod_comp_df.loc[mod_comp_df['scen_name'] == scen, col_plot].values,
                                                mode='lines',
                                                line = dict(color = scen_color),
                                                name=f'{scen}',
                                                legendgroup=f'{scen}',
                                                showlegend = showlegend_TF,
                                                ), row=row, col=col)
            showlegend_TF = True
            fig_sing.add_trace(go.Scatter(x=TT,
                                                y=mod_comp_df.loc[mod_comp_df['scen_name'] == scen, col_plot].values,
                                                mode='lines',
                                                line = dict(color = scen_color),
                                                name=f'{scen}',
                                                legendgroup=f'{scen}',
                                                showlegend = showlegend_TF,
                                                ))
        fig_sing.update_layout(title_text=f"{title} ({col} - {scen})", template='plotly_white', showlegend=True)
        fig_sing.write_html(f'{plot_dir_name}/single_plots/{col_plot}_{scen}.html') if fig_export else None

    fig_comp_scen_legend.update_layout(title_text=f"Comparison {title}", template='plotly_white', showlegend=True) 
    fig_comp_scen_legend.write_html(f'{plot_dir_name}/{agg_file_name}_comparison_simple_legend.html') if fig_export else None

    fig_comp_scen_legend.show() if fig_comp_show else None


### DICE Model

In [ ]:
def DICE_mod(
    scen_name: str                  = 'DICE_model_exog_dyn',            # name of the scenario 
    output_dir_name: str            = 'output',                         # name of the output directory
    exog_vars_filename: str         = 'DICE_exogn_vars_dynamic.csv',    # name of the exogenous variables input file
    solver_method: str              = 'SLSQP',                          # optimization method in scipy.optimize.minimize
    solver_display: bool            = True,                             # display solver output
    max_iter: int                   = 150,                              # maximum number of iterations for optimization
    # general part                                                      # -           
    T_end: int                      = 100,                              # Number of periods
    tstep: int                      = 5,                                # Time step size
    scale1: float                   = 0.0302455265681763,               # Multiplicative scaling coefficient  
    scale2: float                   = -10993.704,                       # Additive scaling coefficient 
    # economic part                                                     # -                                          
    elast_mrg_cons_utility: float   = 1.45,                             # elasticity of marginal utility of consumption
    elast_capt_in_prod: float       = 0.300,                            # elasticity of capital in production function
    social_time_pref_rate: float    = 0.015,                            # Initial rate of social time preference per year
    depr_Capt: float                = 0.1,                              # depreciation rate on capital (per year)
    Capt0: float                    = 223.3,                              # initial world capital 2015 (trillion 2010 USD)
    # emissions part                                                    # -                                   
    ncrb_RFor0: float               = 0.5,                              # 2015 forcings of non-CO2 GHG (Wm-2)
    ncrb_RFor1: float               = 1.0,                              # 2100 forcings of non-CO2 GHG (Wm-2)
    t_ncrb_RFor: int                = 18,                               # Period when non-CO2 forcings switch
    incr_RFor_dbl_crbn: float       = 3.6813,                           # Forcing from doubling CO2 (Wm-2) 2015
    incr_temp_dbl_crb: float        = 3.1,                              # Equilibrium temp impact (oC per doubling CO2)
    a1: float                       = 0,                                # Damage function intercept
    a2: float                       = 0.00236,                          # Damage function quadratic term
    a3: float                       = 2.00,                             # Damage function exponent
    cost2_Abat: float               = 2.6,                              # Theta2 in the model, Exponent of control cost function
    decl_Bstp: float                = 0.025,                            # Initial cost decline backstop  cost per period
    Bstp0: float                    = 550,                              # Cost of backstop technology 2010$ per ton CO2
    ECtr0: float                    = 0.03,                             # initial emissions control rate 2015
    ECtr_lim: float                 = 1.2,                              # Upper limit on control rate after 2150
    ECtr_floor: float               = 0.01,                             # lower limot of control rate for all periods
    t_neg_CEms: int                 = 29,                               # Period when negative emissions are possible
    CStk_atmo0: float               = 851,                              # Initial Concentration in atmosphere 2015 (GtC)
    CStk_atmo1750: float            = 588.0,                            # Initial Concentration in 1750, 
    CStk_ocup0: float               = 460,                              # Initial Concentration in upper strata 2015 (GtC)
    CStk_oclo0: float               = 1740,                             # Initial Concentration in lower strata 2015 (GtC)
    CStk_atmo_eq: float             = 588,                              # Equilibrium concentration atmosphere (GtC)
    CStk_ocup_eq: float             = 360,                              # Equilibrium concentration upper strata (GtC)
    CStk_oclo_eq: float             = 1720,                             # Equilibrium concentration lower strata (GtC)
    b12: float                      = 0.12,                             # carbon cycle transition matrix
    b23: float                      = 0.007,                            # -                               
    Temp_atmo0: float               = 0.85,                             # Initial atmospheric temp change (C from 1900)
    Temp_ocea0: float               = 0.0068,                           # Initial lower stratum temp change (C from 1900)
    c1: float                       = 0.1005,                           # Climate equation coefficient for upper level
    c3: float                       = 0.088,                            # Transfer coefficient upper to lower stratum
    c4: float                       = 0.025,                            # Transfer coefficient for lower level
    ):


    # SETUP + INITALIZATION EXOGENOUS VARIABLES ============================================================
    if True: 
        exog_Vars_DICE = pd.read_csv(f'input/{exog_vars_filename}')# read exogenous variables from input file
        TFPr = exog_Vars_DICE['TFPr'].values
        Labr = exog_Vars_DICE['Labr'].values
        Dcrb = exog_Vars_DICE['Dcrb'].values
        gr_Dcrb = exog_Vars_DICE['gr_Dcrb'].values
        cost1_Abat = exog_Vars_DICE['cost1_Abat'].values  
        CEms_land = exog_Vars_DICE['CEms_land'].values



    # INITIALIZATION PARAMETERS + ENDOGENOUS VARIABLES ============================================================
    if True:
        # General Parameters 
        t = np.arange(1, T_end+1)
        NT = len(t)


        # Economic Part ------------------------------
        social_time_pref_array = 1/((1+social_time_pref_rate)**(tstep*(t-1))) 
        optlSavi = (depr_Capt + 0.004) / (depr_Capt + 0.004*elast_mrg_cons_utility + social_time_pref_rate) * elast_capt_in_prod 

        Capt            = np.zeros(NT)
        Capt[0]         = Capt0
        Prod_gross      = np.zeros(NT)
        Prod_net        = np.zeros(NT)
        Prod            = np.zeros(NT)
        Cons            = np.zeros(NT)
        Cons_pcap       = np.zeros(NT)
        Invs            = np.zeros(NT)
        RI              = np.zeros(NT)
        Util_pcap_cons  = np.zeros(NT)
        Util_cons_disc   = np.zeros(NT)
        

        # Emissions Part ------------------------------
        # radiative forcing 
        ncrb_RFor = np.full(NT, ncrb_RFor0)
        ncrb_RFor[0:t_ncrb_RFor] = ncrb_RFor[0:t_ncrb_RFor] + (1/(t_ncrb_RFor-1))*(ncrb_RFor1-ncrb_RFor0)*(t[0:t_ncrb_RFor]-1)
        ncrb_RFor[t_ncrb_RFor:NT] = ncrb_RFor[t_ncrb_RFor:NT] + (ncrb_RFor1-ncrb_RFor0)
        RFor = np.zeros(NT)

        #  backstop technology price
        Bstp = Bstp0 * (1-decl_Bstp)**(t-1)  # backstop price

        # carbon cycle transition matrix
        b11 = 1 - b12 
        b21 = b12*(CStk_atmo_eq/CStk_ocup_eq)
        b22 = 1 - b21 - b23
        b32 = b23*(CStk_ocup_eq/CStk_oclo_eq)
        b33 = 1 - b32

        # rest
        Damg_frac       = np.zeros(NT)
        Damg            = np.zeros(NT)
        Abat_cost       = np.zeros(NT)
        Abat_MC         = np.zeros(NT)
        Cpri            = np.zeros(NT)
        CEms            = np.zeros(NT)
        CEms_indu       = np.zeros(NT)
        CEms_indu_cum   = np.zeros(NT)
        CEms_land_cum   = np.zeros(NT)
        CEms_tot_cum    = np.zeros(NT)
        CStk_atmo       = np.zeros(NT)
        CStk_atmo[0]    = CStk_atmo0
        CStk_ocup       = np.zeros(NT)
        CStk_ocup[0]    = CStk_ocup0
        CStk_oclo       = np.zeros(NT)
        CStk_oclo[0]    = CStk_oclo0
        Temp_atmo       = np.zeros(NT)
        Temp_atmo[0]    = Temp_atmo0
        Temp_ocea       = np.zeros(NT)
        Temp_ocea[0]    = Temp_ocea0



    # ENDOGENOUS VARIABLES DEFINITION ============================================================

    # Economic Part ------------------------------
    if True:
        def fProd_gross(iTFPr, iLabr, iCapt, idx):
            return iTFPr[idx] * ((iLabr[idx]/1000) ** (1 - elast_capt_in_prod)) * (iCapt[idx] ** elast_capt_in_prod)

        def fProd_net(iProd_gross, iDamg_frac, idx):
            return iProd_gross[idx]    *  (1 - iDamg_frac[idx])  

        def fProd(iProd_net, iAbat_cost, idx):
            return iProd_net[idx]  -  iAbat_cost[idx]
        
        def fCons(iProd, iInvs, idx):
            return iProd[idx] - iInvs[idx]

        def fCons_pcap(iCons, iLabr, idx):
            return 1000 * iCons[idx] / iLabr[idx]

        def fInvs(iSavi, iProd, idx):
            return iSavi[idx] * iProd[idx]

        def fCapt(iCapt, iInvs, idx):
            if (idx == 0):
                return Capt0
            else:
                return (1-depr_Capt)**tstep * iCapt[idx-1] + tstep * iInvs[idx-1]

        def fRI(iCons_pcap, idx):
            if (idx == 0):
                return 0
            else: 
                return (1 + social_time_pref_rate) * (iCons_pcap[idx] / iCons_pcap[idx-1]) ** (elast_mrg_cons_utility/tstep) - 1

        def fUtil_pcap_cons(iCons, iLabr, idx):
            return ((iCons[idx] * 1000 / iLabr[idx]) ** (1 - elast_mrg_cons_utility) -1) / (1 - elast_mrg_cons_utility) - 1 

        def fUtil_cons_disc(iUtil_pcap_cons, iLabr, idx):
            return iUtil_pcap_cons[idx] * iLabr[idx] * social_time_pref_array[idx]

        def fUTILITY(iUtil_cons_disc, resUtility):
            resUtility[0] = tstep * scale1 * np.sum(iUtil_cons_disc) + scale2



    # Emissions Part ------------------------------
    if True:
        def fCEms(iCEms_indu, iCEms_land, idx):
            return iCEms_indu[idx] + iCEms_land[idx]

        def fCEms_indu(iProd_gross, iECtr, iDcrb, idx):
            return iDcrb[idx] * iProd_gross[idx] * (1 - iECtr[idx])
        
        def fCEms_indu_cum(iCEms_indu_cum, iCEms_indu, idx):
            if (idx == 0):
                return iCEms_indu[idx] * (5 / 3.666)
            else:
                return iCEms_indu_cum[idx-1] + iCEms_indu[idx] * (5 / 3.666)
        
        def fCEms_land_cum(iCEms_land_cum, iCEms_land, idx):
            if (idx == 0):
                return iCEms_land[idx] * (5 / 3.666)
            else:
                return iCEms_land_cum[idx-1] + iCEms_land[idx] * (5 / 3.666)
                
        def fCEms_tot_cum(iCEms_indu_cum, iCEms_land_cum, idx):
            return iCEms_indu_cum[idx] + iCEms_land_cum[idx] 

        def fRFor(iCStk_atmo, idx):
            return incr_RFor_dbl_crbn * np.log(iCStk_atmo[idx] / CStk_atmo1750) / np.log(2) + ncrb_RFor[idx]

        def fDamg_frac(iTemp_atmo, idx):
            return a1 * iTemp_atmo[idx] + a2*iTemp_atmo[idx]**a3

        def fDamg(iProd_gross, iDamg_frac, idx):
            return iProd_gross[idx] * iDamg_frac[idx]

        def fAbat_cost(iProd_gross, iECtr, icost1_Abat, idx):
            return iProd_gross[idx] * icost1_Abat[idx] * iECtr[idx] ** cost2_Abat

        def fAbat_MC(iECtr, idx):
            return Bstp[idx] * iECtr[idx]**(cost2_Abat-1)

        def fCpri(iECtr, idx):
            return Bstp[idx] * iECtr[idx]**(cost2_Abat-1)


        def fCStk_atmo(iCStk_atmo, iCStk_ocup, iCEms, idx):
            if (idx == 0):
                return CStk_atmo0
            else:
                return iCStk_atmo[idx-1]*b11 + iCStk_ocup[idx-1]*b21 + iCEms[idx-1]*5/3.666

        def fCStk_ocup(iCStk_atmo, iCStk_ocup, iCStk_oclo, idx):
            if (idx == 0):
                return CStk_ocup0
            else:
                return iCStk_atmo[idx-1]*b12 + iCStk_ocup[idx-1]*b22 + iCStk_oclo[idx-1]*b32

        def fCStk_oclo(iCStk_ocup, iCStk_oclo, idx):
            if (idx == 0):
                return CStk_oclo0
            else:
                return iCStk_oclo[idx-1]*b33 + iCStk_ocup[idx-1]*b23

        def fTemp_atmo(iTemp_atmo, iRFor, iTemp_ocea, idx):
            if (idx == 0):
                return Temp_atmo0
            else:
                return iTemp_atmo[idx-1] + c1*(iRFor[idx] - (incr_RFor_dbl_crbn/incr_temp_dbl_crb)*iTemp_atmo[idx-1] - c3*(iTemp_atmo[idx-1] - iTemp_ocea[idx-1])) 

        def fTemp_ocea(iTemp_atmo, iTemp_ocea, idx):
            if (idx == 0):
                return Temp_ocea0
            else:
                return iTemp_ocea[idx-1] + c4*(iTemp_atmo[idx-1] - iTemp_ocea[idx-1])



    # DECISION VARIABLES + BOUNDS DEFINITION ============================================================
    if True: 
        # creating arrays for upper and lower bound for decision variables 
        # which are later combined into the list of tuple format that we know.

        # Emission Control Rate ----------------------------------------
        ECtr_lo = np.full(NT, ECtr_floor)        
        ECtr_up = np.full(NT, ECtr_lim)
        ECtr_up[0:t_neg_CEms] = 1  
        
        # hard bound first period
        ECtr_lo[0] = ECtr0      
        ECtr_up[0] = ECtr0  
        ECtr_lo[ECtr_lo==ECtr_up] = 0.99999*ECtr_lo[ECtr_lo==ECtr_up]  # coding trick: set bounds to basically a single value, still allowing minimal range not to constrain solver to infeasability
        
        # combine bound arrays to a list of tuples
        bnds_ECtr=[]
        for i in range(NT):
            bnds_ECtr.append((ECtr_lo[i],ECtr_up[i]))

        # starting values
        ECtr_start = 0.99 * ECtr_up                                       # starting values just below upper bound
        ECtr_start[ECtr_start < ECtr_lo] = ECtr_lo[ECtr_start < ECtr_lo]  # readjust starting values within bounds, if necessary
        ECtr_start[ECtr_start > ECtr_up] = ECtr_up[ECtr_start > ECtr_up]


        # Savings Rate ----------------------------------------
        Savi_lo = np.full(NT, 0.1)    
        Savi_up = np.full(NT, 0.9)

        # hard bound last 10 periods
        lag10 = t > NT - 10 # indicator last 10 periods
        Savi_lo[lag10] = optlSavi
        Savi_up[lag10] = optlSavi
        Savi_lo[Savi_lo==Savi_up] = 0.99999*Savi_lo[Savi_lo==Savi_up]       # coding trick: set bounds to basically a single value, still allowing minimal range not to constrain solver to infeasability

        # combine bound arrays to a list of tuples
        bnds_Savi=[]
        for i in range(NT):
            bnds_Savi.append((Savi_lo[i],Savi_up[i]))

        # arbitrary starting values
        Savi_start = np.full(NT, 0.2)                                  
        Savi_start[Savi_start < Savi_lo] = Savi_lo[Savi_start < Savi_lo]
        Savi_start[Savi_start > Savi_up] = Savi_up[Savi_start > Savi_up]



    # OBJECTIVE ============================================================
    def fOBJ(x, sign, 
                iTFPr, 
                iLabr, 
                iDcrb, 
                icost1_Abat, 
                iCEms_land, 
                iCapt, 
                iProd_gross, 
                iProd_net, 
                iProd, 
                iInvs, 
                iCons, 
                iCons_pcap, 
                iRI, 
                iUtil_pcap_cons, 
                iUtil_cons_disc, 
                iCEms_indu, 
                iCEms, 
                iCEms_indu_cum, 
                iCEms_land_cum, 
                iCEms_tot_cum, 
                iCStk_atmo, 
                iCStk_oclo, 
                iCStk_ocup, 
                iRFor, 
                iTemp_atmo, 
                iTemp_ocea, 
                iDamg_frac, 
                iDamg, 
                iAbat_cost, 
                iAbat_MC, 
                iCpri, 
                iNT,
            ):
        
        iECtr = x[0:NT]
        iSavi = x[NT:(2*NT)]
        
        for i in range(iNT):
            iCapt[i]            = fCapt(iCapt, iInvs, i)
            iProd_gross[i]      = fProd_gross(iTFPr, iLabr, iCapt, i)
            iCEms_indu[i]       = fCEms_indu(iProd_gross, iECtr, iDcrb, i)
            iCEms[i]            = fCEms(iCEms_indu, iCEms_land, i)            
            iCEms_indu_cum[i]   = fCEms_indu_cum(iCEms_indu_cum, iCEms_indu, i)
            iCEms_land_cum[i]   = fCEms_land_cum(iCEms_land_cum, iCEms_land, i)
            iCEms_tot_cum[i]    = fCEms_tot_cum(iCEms_indu_cum, iCEms_land_cum, i)
            iCStk_atmo[i]       = fCStk_atmo(iCStk_atmo, iCStk_ocup, iCEms, i)
            iCStk_oclo[i]       = fCStk_oclo(iCStk_ocup, iCStk_oclo, i)
            iCStk_ocup[i]       = fCStk_ocup(iCStk_atmo, iCStk_ocup, iCStk_oclo, i)
            iRFor[i]            = fRFor(iCStk_atmo, i)
            iTemp_atmo[i]       = fTemp_atmo(iTemp_atmo, iRFor, iTemp_ocea, i)
            iTemp_ocea[i]       = fTemp_ocea(iTemp_atmo, iTemp_ocea, i)
            iDamg_frac[i]       = fDamg_frac(iTemp_atmo, i)
            iDamg[i]            = fDamg(iProd_gross, iDamg_frac, i)
            iAbat_cost[i]       = fAbat_cost(iProd_gross, iECtr, icost1_Abat, i)
            iAbat_MC[i]         = fAbat_MC(iECtr, i)
            iCpri[i]            = fCpri(iECtr, i)
            iProd_net[i]        = fProd_net(iProd_gross, iDamg_frac, i)
            iProd[i]            = fProd(iProd_net, iAbat_cost, i)
            iInvs[i]            = fInvs(iSavi, iProd, i)
            iCons[i]            = fCons(iProd, iInvs, i)
            iCons_pcap[i]       = fCons_pcap(iCons, iLabr, i)
            iUtil_pcap_cons[i]  = fUtil_pcap_cons(iCons, iLabr, i)
            iUtil_cons_disc[i]  = fUtil_cons_disc(iUtil_pcap_cons, iLabr, i)
            iRI[i]              = fRI(iCons_pcap, i)

        resUtility = np.zeros(1)
        fUTILITY(iUtil_cons_disc, resUtility)

        return sign * resUtility[0]
   
            


    # OPTIMIZATION ============================================================
    start_time = datetime.datetime.now()
    print(f'\n-- START: {scen_name} -- {start_time} {30*"-"}')

    x_start = np.concatenate([ECtr_start,Savi_start])
    bnds = bnds_ECtr + bnds_Savi  
    cnstr = []

    try:
        result = opt.minimize(fOBJ, x_start, 
                            args = (-1.0,
                                    TFPr,           
                                    Labr,           
                                    Dcrb,           
                                    cost1_Abat,     
                                    CEms_land,      
                                    Capt,           
                                    Prod_gross,     
                                    Prod_net,       
                                    Prod,           
                                    Invs,           
                                    Cons,           
                                    Cons_pcap,      
                                    RI,             
                                    Util_pcap_cons, 
                                    Util_cons_disc, 
                                    CEms_indu,      
                                    CEms,           
                                    CEms_indu_cum,  
                                    CEms_land_cum,  
                                    CEms_tot_cum,   
                                    CStk_atmo,      
                                    CStk_oclo,      
                                    CStk_ocup,      
                                    RFor,           
                                    Temp_atmo,      
                                    Temp_ocea,      
                                    Damg_frac,      
                                    Damg,           
                                    Abat_cost,      
                                    Abat_MC,        
                                    Cpri,           
                                    NT,             
                                    ),
                            method=solver_method ,
                            bounds = tuple(bnds),
                            constraints = cnstr,
                            options={'disp': solver_display ,'maxiter': max_iter})
    except Exception as e:
        print(f"An error occurred during optimization: {scen_name} - {e}")


    # EXPORT ============================================================
    if result.success:
        tuples_to_plot = (
            (result.x[0:NT],        'ECtr'                   ),
            (result.x[NT:(2*NT)],   'Savi'                   ),
            (TFPr,                  'TFPr'                   ),
            (Labr,                  'Labr'                   ),
            (Dcrb,                  'Dcrb'                   ),
            (gr_Dcrb,               'gr_Dcrb'                ),
            (cost1_Abat,            'cost1_Abat'             ),
            (CEms_land,             'CEms_land'              ),
            (Capt,                  'Capt'                   ),
            (Prod_gross,            'Prod_gross'             ),
            (Prod_net,              'Prod_net'               ),
            (Prod,                  'Prod'                   ),
            (Invs,                  'Invs'                   ),
            (Cons,                  'Cons'                   ),
            (Cons_pcap,             'Cons_pcap'              ),
            (Util_pcap_cons,        'Util_pcap_cons'         ),
            (Util_cons_disc,        'Util_cons_disc'         ),
            (RI,                    'RI'                     ),
            (CEms,                  'CEms'                   ),
            (CEms_indu,             'CEms_indu'              ),
            (CEms_land_cum,         'CEms_land_cum'          ),
            (CEms_indu_cum,         'CEms_indu_cum'          ),
            (CEms_tot_cum,          'CEms_tot_cum'           ),
            (RFor,                  'RFor'                   ),
            (ncrb_RFor,             'ncrb_RFor'              ),
            (Abat_cost,             'Abat_cost'              ),
            (Abat_MC,               'Abat_MC'                ),
            (Cpri,                  'Cpri'                   ),
            (CStk_atmo,             'CStk_atmo'              ),
            (CStk_ocup,             'CStk_ocup'              ),
            (CStk_oclo,             'CStk_oclo'              ),
            (Temp_atmo,             'Temp_atmo'              ),
            (Temp_ocea,             'Temp_ocea'              ),
            (Damg_frac,             'Damg_frac'              ),
            (Damg,                  'Damg'                   ),
            (social_time_pref_array,'social_time_pref_array' ),
            )

        results_df = pd.DataFrame({'scen_name': scen_name, 't': t, 't_year': 2000 + t*tstep})
        for var_object, col_name in tuples_to_plot:
            results_df[col_name] = var_object
        os.makedirs(f'{output_dir_name}', exist_ok=True) 
        results_df.to_csv(f'{output_dir_name}/{scen_name}_results_df.csv', index=False)


    # settings .txt file to store scenario parameters
    with open(f'{output_dir_name}/{scen_name}_settings.txt', 'w') as f:
        f.write(f'{50*"#"}\nscen_name: {scen_name}\n{50*"#"}\n\n')
        input_param_names = ['scen_name', 'exog_vars_filename', 'solver_method', 'T_end', 'tstep', 'scale1', 'scale2', 
                            'elast_mrg_cons_utility', 'elast_capt_in_prod', 'social_time_pref_rate', 'depr_Capt', 'Capt0', 
                            'ncrb_RFor0', 'ncrb_RFor1', 't_ncrb_RFor', 'incr_RFor_dbl_crbn', 'incr_temp_dbl_crb', 'a1', 'a2', 'a3', 
                            'cost2_Abat', 'decl_Bstp', 'Bstp0', 'ECtr0', 'ECtr_lim', 'ECtr_floor', 't_neg_CEms', 'CStk_atmo0', 'CStk_atmo1750', 
                            'CStk_ocup0', 'CStk_oclo0', 'CStk_atmo_eq', 'CStk_ocup_eq', 'CStk_oclo_eq', 'b12', 'b23', 'Temp_atmo0', 
                            'Temp_ocea0', 'c1', 'c3', 'c4', ]
        
        for key, value in locals().items():
            if key in input_param_names:
                f.write(f'{key:<30}: {value:<900}\n')
        f.write(f'\n{50*"#"}\n')


    # timer .txt file for runtime
    end_time = datetime.datetime.now()
    with open(f'{output_dir_name}/{scen_name}_timer.txt', 'w') as f:
        f.write(f'start:  {scen_name} \t{start_time}\n')
        f.write(f'end:    {scen_name} \t{end_time}\n')
        f.write(f'runtime {scen_name}: \t{end_time  - start_time} (h:m:s.ms)\n\n')


### Runs

In [182]:
# # ATTENTION: This is a short line of code removing all directories with the name "output" in them.
# #            It is handy to clean up your model outputs and visualize only the latest runs using plot_comparison()        
old_dir = glob.glob('output_nb*')
for dir in old_dir:
    shutil.rmtree(f'{dir}/')

subdir_path = 'output_default'
DICE_mod(scen_name='DICE_default',
        output_dir_name = subdir_path, 
        )
plot_comparison(output_dir_name = 'output_default', 
                plot_dir_name='plots_default', 
                agg_file_name= '0_plot_default', 
                title= 'DICE Comparison Economic & Emission Variables', 
                fig_indv_show=False, 
                fig_export = False, 
                )

subdir_path = 'output_deprCapt_scenarios'
DICE_mod(scen_name='DICE_default',
        output_dir_name = subdir_path, 
        )
DICE_mod(scen_name='DICE_deprCapt0.05',
        output_dir_name = subdir_path, 
        depr_Capt= 0.05
        )
DICE_mod(scen_name='DICE_deprCapt0.2',
        output_dir_name = subdir_path, 
        depr_Capt= 0.2
        )
plot_comparison(output_dir_name = 'output_deprCapt_scenarios', 
                plot_dir_name='plots_deprCapt_scenarios', 
                agg_file_name= '0_plot_deprCapt_comparison', 
                title= 'DICE Comparison Economic & Emission Variables', 
                fig_indv_show=False
                )

subdir_path = 'output_Capt0_scenarios'
DICE_mod(scen_name='DICE_default',
        output_dir_name = subdir_path, 
        )
DICE_mod(scen_name='DICE_Capt22k',
        output_dir_name = subdir_path, 
        Capt0=22330
        )

plot_comparison(output_dir_name = 'output_Capt0_scenarios', 
                plot_dir_name='plots_Capt0_scenarios', 
                agg_file_name= '0_plot_Capt0_comparison', 
                title= 'DICE Comparison Economic & Emission Variables', 
                )


-- START: DICE_default -- 2025-04-29 11:35:49.986767 ------------------------------


c:\Models\ResEcon_MASTER\.venv_resecon_master\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: -4517.706169131006
            Iterations: 95
            Function evaluations: 19234
            Gradient evaluations: 95

-- START: DICE_deprCapt0.05 -- 2025-04-29 11:38:05.445841 ------------------------------


c:\Models\ResEcon_MASTER\.venv_resecon_master\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: -5070.197260787303
            Iterations: 90
            Function evaluations: 18217
            Gradient evaluations: 90

-- START: DICE_deprCapt0.2 -- 2025-04-29 11:40:34.767815 ------------------------------


...


-- START: DICE_default -- 2025-04-29 11:43:56.757576 ------------------------------


c:\Models\ResEcon_MASTER\.venv_resecon_master\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: -4517.706169131006
            Iterations: 95
            Function evaluations: 19234
            Gradient evaluations: 95

-- START: DICE_Capt22k -- 2025-04-29 11:46:49.806935 ------------------------------


c:\Models\ResEcon_MASTER\.venv_resecon_master\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: -6322.001047556745
            Iterations: 98
            Function evaluations: 19819
            Gradient evaluations: 98
